In [1]:
#!pip install openpyxl
import pandas as pd


%load_ext autoreload
%autoreload 2
def d(l):
    pd.set_option('display.max_colwidth', l)

d(10)

In [3]:
science=pd.read_excel("2025-04-02 METADATA_ science-nature_V2.xlsx",sheet_name="METADATA Science")[["ID","Date","Type","Doi"]]
nature=pd.read_excel("2025-04-02 METADATA_ science-nature_V2.xlsx",sheet_name="METADATA Nature")[["ID","Date","Type","Doi"]]

df_old=pd.read_excel("Single item merged file.xlsx", usecols=['ID_Paper','Type Recoded','Type','Date YYMMDD']) # File from Fred & Auste : Better data than in the other files (in these columns)


In [4]:
GPT_retract=pd.read_json("2025-04-03 full data with GPT4o-mini_response.json")
GPT_retract["ID"]=GPT_retract.index

In [5]:
# Does this make sense ??

from ast import literal_eval

column_name_duplicated="text duplicated"

# Create a "duplicated" column by checking for duplicates in the processed text

GPT_retract[column_name_duplicated] = GPT_retract.filtered_texts.apply(lambda x: "".join(x)).str[:100].duplicated(keep=False)
x=GPT_retract[GPT_retract[column_name_duplicated]]

x["ID"]

Science17     Science17
Science158    Scienc...
Science446    Scienc...
Science394    Scienc...
Science397    Scienc...
                ...    
Nature1699    Nature...
Nature1692    Nature...
Nature1687    Nature...
Nature1686    Nature...
Nature573     Nature573
Name: ID, Length: 838, dtype: object

In [9]:
nature_relevance=pd.read_excel("Nature tally.xlsx")
nature_relevance.columns=nature_relevance.columns.str.replace("3.eliminated","3. eliminated")
nature_relevance.value_counts()

science_relevance=pd.read_excel("science tally.xlsx",sheet_name="final tally")
science_relevance.columns=science_relevance.columns.str.replace("retrieved ","retrieved").str.replace("3.eliminated","3. eliminated")
science_relevance.value_counts()


elimination_columns=['1. eliminated because no pdf doc',
       '2. eliminated because no occurance of retract',
       '3. eliminated because editorial retraction',
       '4. eliminated because book reviews, etc.',
       '5. eliminated because scanned with other relevant doc, duplicate or other technical issue',
       '6. eliminated because context/meaning']



In [11]:
science_eliminated=science_relevance[elimination_columns].apply(lambda x: x.str.title().value_counts())
print("Sum: ",science_eliminated.sum(axis=1))
print("\n\n",science_relevance["8. retained for analysis"].str.title().str.replace("X","No").value_counts())


nature_eliminated=nature_relevance[elimination_columns].apply(lambda x: x.str.title().value_counts())
print("Sum: ",nature_eliminated.sum(axis=1))
print("\n\n",nature_relevance["8. retained for analysis"].str.title().value_counts())


Sum:  Yes    1018
dtype: int64


 8. retained for analysis
No     1018
Yes     377
Name: count, dtype: int64
Sum:  Yes    1178
dtype: int64


 8. retained for analysis
No     1178
Yes     656
Name: count, dtype: int64


In [13]:
# all files

df_relevance=pd.concat([science_relevance,nature_relevance])

#add GPT_retract
df_relevance=pd.merge(df_relevance,GPT_retract,left_on="ID",right_on="ID",how="outer")

df_relevance.iloc[:,3:]=df_relevance.iloc[:,3:].replace({'yes': True, 'Yes': True,"YES":True})
#df_relevance["2. eliminated because no occurance of retract"]

df_relevance[elimination_columns].apply(lambda x: x.value_counts()).T

,True
1. eliminated because no pdf doc,385
2. eliminated because no occurance of retract,518
3. eliminated because editorial retraction,61
"4. eliminated because book reviews, etc.",200
"5. eliminated because scanned with other relevant doc, duplicate or other technical issue",226
6. eliminated because context/meaning,806


In [15]:

df_relevance["retraction_revision"]=(df_relevance["2. eliminated because no occurance of retract"]==True) & (df_relevance["has_retraction"]==True)

x=df_relevance.dropna(subset="has_retraction")
x[elimination_columns].apply(lambda x: x.value_counts()).T

/var/folders/xb/hcrlz1px0q5byjc3wrssw2lm0000gn/T/ipykernel_8233/3472833730.py:4: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  x[elimination_columns].apply(lambda x: x.value_counts()).T


,True
1. eliminated because no pdf doc,NaN
2. eliminated because no occurance of retract,111.0
3. eliminated because editorial retraction,60.0
"4. eliminated because book reviews, etc.",198.0
"5. eliminated because scanned with other relevant doc, duplicate or other technical issue",226.0
6. eliminated because context/meaning,806.0


In [17]:
df_relevance["PDF retrieved"]=df_relevance["PDF retrieved"].replace("Yes but no OCR",True)
df_relevance.loc[df_relevance["PDF retrieved"]!=True,"PDF retrieved"]=False
df_relevance["PDF retrieved"].value_counts()
df_relevance["has_retraction"].replace(1,True)

0       True
1       True
2       True
3       True
4        NaN
        ... 
3224     NaN
3225     NaN
3226    True
3227    True
3228     NaN
Name: has_retraction, Length: 3229, dtype: object

In [19]:
df=pd.concat([science,nature])
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
from datetime import datetime
df['Year']=df['Date'].apply(lambda x: x.year if isinstance(x,datetime) else x)
df["journal"]=df.ID.str.replace(r'\d+', '', regex=True)

# Recode the 'Type' column
df['Type Recoded'] = df['Type'].replace({
    'Perspectives and commentary': 'Contribution',
    'Comments & Opinion': 'Contribution',
    'Books & Arts': 'Book Reviews',
    'News': 'Contribution',
    'Correspondence': 'Journalism',
    'Editorial': 'Journalism',
    'other': 'Journalism',
    'News & Views': 'Journalism',
    'Special Features': 'Other',
    'Not comment/opinion': 'Other',
    'Meeting report': 'Other',
    'Research Highlights': 'Other',
    None: 'Other'
})


df_old['Date'] = pd.to_datetime(df_old['Date YYMMDD'], errors='coerce')
df.dropna(subset="ID",inplace=True)
df=pd.merge(df,df_old,right_on="ID_Paper",left_on="ID",how="outer")


In [21]:
# some metadata was recoded: 

#See
#df[df["Date_x"]!=df["Date_y"]][["ID","Date_x","Date_y"]].dropna()
#df[df["Type_x"]!=df["Type_y"]][["ID","Type_x","Type_y"]].dropna()


# Action: I take the recoded ones!
df[df["Type_x"]!=df["Type_y"]][["ID","Type_x","Type_y"]].dropna()

# Update Type_x only where Type_y has a value
df["Type_x"] = df["Type_x"].where(df["Type_y"].isna(), df["Type_y"])
df["Type Recoded_x"] = df["Type Recoded_x"].where(df["Type Recoded_y"].isna(), df["Type Recoded_y"])
df["Date_x"] = df["Date_x"].where(df["Date_y"].isna(), df["Date_y"])

df["confirmed_2024"]=df.ID_Paper.isna()==False
df.drop(columns=["Type_y","Date_y",'Type Recoded_y','Date YYMMDD',"ID_Paper"],inplace=True)
df.columns = df.columns.str.replace('_x', '', regex=False)

In [23]:
df["Year"]=df["Date"].dt.year
df["7. eliminated because before 1960"]=df["Date"]<pd.to_datetime("01-01-1960")

In [25]:
df_final=pd.merge(df,df_relevance,on=["Doi","ID"],suffixes=[None,"_y"])


## Manually add two more articles: Nature507b and Nature454b

#--> These articles were missing in the initial meta-data overview and only later found as seperate articles within a PDF.
#How do I add them? 
#- I increase the overall number of articles and the final number of articles by 2 and take these +2 through all other steps.
#- I split the text in the textfile to make it two different articles.


df_final.index=df_final.ID
df_final.loc['Nature507b',:]=df_final.loc['Nature507',:]
df_final.loc['Nature454b',:]=df_final.loc['Nature454',:]
df_final.ID=df_final.index

df_final['1. eliminated because no pdf']=df_final['PDF retrieved']==False
df_final['2. eliminated because corrupt file or no keyword']=df_final["text"].isna()


df_final.shape

(3231, 45)

In [27]:
df_final.columns

Index(['ID', 'Date', 'Type', 'Doi', 'Year', 'journal', 'Type Recoded',
       'confirmed_2024', '7. eliminated because before 1960', 'Title',
       'PDF retrieved', 'If no, why?', 'Appears in first context document?',
       'If no, why?.1', 'Appears in second context document?',
       '1. eliminated because no pdf doc',
       '2. eliminated because no occurance of retract',
       '3. eliminated because editorial retraction',
       '4. eliminated because book reviews, etc.',
       '5. eliminated because scanned with other relevant doc, duplicate or other technical issue',
       '6. eliminated because context/meaning', '7. eliminated because….',
       '8. retained for analysis', 'path', 'folder', 'journal_y', 'texts',
       'labels', 'retraction_counts', 'has_retraction', 'retract_tokens',
       'has_retraction_label', 'filtered_texts', 'neighboring_texts',
       'texts_until_section_header', 'text', 'relevant', 'retraction_context',
       'retraction_explanation', 'category

In [29]:
columns_to_check=['1. eliminated because no pdf',
                  '2. eliminated because corrupt file or no keyword',
                  '3. eliminated because editorial retraction',
       '4. eliminated because book reviews, etc.',
       '5. eliminated because scanned with other relevant doc, duplicate or other technical issue',
       '6. eliminated because context/meaning', "7. eliminated because before 1960"]



In [31]:
#remove text duplicates
#df_final["9. retained for analysis (revised)"]=(df_final["has_retraction"]==True) & (~df_final["3-6 eliminated"]==True) & (~df_final[column_name_duplicated].dropna()==True)

# keep text duplicates
df_final["PDF retrieved"]=df_final["PDF retrieved"].apply(lambda x: True if x=="Yes but no OCR" else x)


df_final['1-7 eliminated'] = df_final[columns_to_check].apply(lambda row: row.any(), axis=1)


df_final["9. retained for analysis (revised)"]=(df_final["has_retraction"]==True) & (~df_final["1-7 eliminated"]==True)
df_final["8. retained for analysis"]=df_final["8. retained for analysis"].replace({'yes': True, 'Yes': True, 'YES': True})
df_final["8. retained for analysis"]=df_final["8. retained for analysis"].replace("No",False).replace("no",False).replace("x",False)
df_final.loc[df_final['PDF retrieved']!=True,'PDF retrieved']=False
print("before manual check of new files")
df_final["9. retained for analysis (revised)"].value_counts()



before manual check of new files


/var/folders/xb/hcrlz1px0q5byjc3wrssw2lm0000gn/T/ipykernel_8233/826376612.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final["8. retained for analysis"]=df_final["8. retained for analysis"].replace("No",False).replace("no",False).replace("x",False)


9. retained for analysis (revised)
False    2166
True     1065
Name: count, dtype: int64

In [33]:
# Manually change the status of 3 articles which are duplications

df_final.loc[['Science231',"Science755","Science200"],'5. eliminated because scanned with other relevant doc, duplicate or other technical issue']=True
df_final.loc[['Science231',"Science755","Science200"],'8. retained for analysis']=False
df_final.loc[['Science231',"Science755","Science200"],'relevant']=False
df_final.loc[['Science231',"Science755","Science200"],"9. retained for analysis (revised)"]=False





In [35]:
# Manually annotate new files

# Compare the data lists and create a file with new files (from re-doing OCR & searching "retraction"
df_final["NEW in the analysis"]=(df_final["8. retained for analysis"]==False) & (df_final["9. retained for analysis (revised)"]==True) & (df_final["7. eliminated because before 1960"]==False)
df_final["REMOVED from the analysis"]=(df_final["8. retained for analysis"]==True) & (df_final["9. retained for analysis (revised)"]==False) & (df_final["7. eliminated because before 1960"]==False)
df_final[df_final["NEW in the analysis"]].to_excel("2025-04-25 new articles for manual annotation of retraction context.xlsx")


# Load the manually annotated articles
include_manually_annotated = pd.read_excel("2025-04-26 new articles manually annotated.xlsx")

# Filter IDs based on the manual check
new_exclude = include_manually_annotated[include_manually_annotated["Manual Check CONTEXT INCLUDE"] != True]["ID"]
new_include = include_manually_annotated[include_manually_annotated["Manual Check CONTEXT INCLUDE"] == True]["ID"]

# Update the df_final DataFrame
df_final.loc[df_final['ID'].isin(new_exclude), '6. eliminated because context/meaning'] = True
df_final.loc[df_final['ID'].isin(new_exclude), "9. retained for analysis (revised)"]=False


df_final.loc[df_final['ID'].isin(new_include), ['2. eliminated because corrupt file or no keyword',
                                                '3. eliminated because editorial retraction',
                                                '4. eliminated because book reviews, etc.',
                                                '5. eliminated because scanned with other relevant doc, duplicate or other technical issue',
                                                '6. eliminated because context/meaning',
                                                '3-7 eliminated']] = False
df_final.loc[df_final['ID'].isin(new_include),"9. retained for analysis (revised)"]=True

df_final["9. retained for analysis (revised)"].value_counts()



9. retained for analysis (revised)
False    2215
True     1016
Name: count, dtype: int64

In [222]:

df_final.to_excel("2025-06-07 all meta data.xlsx")
df_final.to_json("2025-06-07 all meta data.json")

In [224]:
# Drop rows with missing values in the specified column and create a new DataFrame
bad_context = df_final.dropna(subset=['6. eliminated because context/meaning'])
# Select the relevant column from the new DataFrame
bad_context['6. eliminated because context/meaning']
#bad_context[["ID","Doi","Title","Date"]+elimination_columns+['7. eliminated because before 1960']].sort_values('7. eliminated because before 1960').to_excel("2025-04-03_context eliminations.xlsx")

ID
Nature10      True
Nature1000    True
Nature1005    True
Nature1007    True
Nature101     True
              ... 
Science987    True
Science992    True
Science993    True
Science994    True
Science998    True
Name: 6. eliminated because context/meaning, Length: 871, dtype: object

In [37]:
withPDF=df_final[df_final["1. eliminated because no pdf"]!=True]
print(len(withPDF))

withKeyword=withPDF[withPDF["2. eliminated because corrupt file or no keyword"]!=True]
print(len(withKeyword))

NoEditR=withKeyword[withKeyword["3. eliminated because editorial retraction"]!=True]
print(len(NoEditR))

NoBookR=NoEditR[NoEditR["4. eliminated because book reviews, etc."]!=True]
print(len(NoBookR))

CorrectContext=NoBookR[NoBookR["6. eliminated because context/meaning"]!=True]
print(len(CorrectContext))

No_Duplicated=CorrectContext[CorrectContext["text duplicated"]==False]

No_DuplicatedProblem=CorrectContext[CorrectContext["text duplicated"]==True]


NoTechIssue=No_Duplicated[No_Duplicated["5. eliminated because scanned with other relevant doc, duplicate or other technical issue"]!=True]

No_DuplicatedTechIssue=No_Duplicated[No_Duplicated["5. eliminated because scanned with other relevant doc, duplicate or other technical issue"]==True]
print(len(No_DuplicatedTechIssue))



After1960=NoTechIssue[NoTechIssue["7. eliminated because before 1960"]!=True]
No_DuplicatedTechIssueAfter1960=No_DuplicatedTechIssue[No_DuplicatedTechIssue["7. eliminated because before 1960"]!=True]

print(len(No_Duplicated),

      len(No_DuplicatedTechIssue),
      len(No_DuplicatedTechIssueAfter1960))

print(len(After1960))

strange=After1960[After1960["9. retained for analysis (revised)"]!=True]


2846
2436
2377
2179
1325
23
967 23 20
891


In [228]:
No_DuplicatedTechIssueAfter1960.to_excel("2025-04-29 WhatTechIssue.xlsx")

In [39]:
#Reorganize Column

df_final2=df_final[['ID', 'Doi', 'journal' ,'Title', 'Date', 'Year', 'Type', 'Type Recoded',
       'confirmed_2024',
       'PDF retrieved','has_retraction',
       '1. eliminated because no pdf',
       '2. eliminated because corrupt file or no keyword',
       '3. eliminated because editorial retraction',
       '4. eliminated because book reviews, etc.',
       '5. eliminated because scanned with other relevant doc, duplicate or other technical issue',
       '6. eliminated because context/meaning', '7. eliminated because before 1960', '3-7 eliminated',
       '8. retained for analysis',"9. retained for analysis (revised)"]]



In [41]:
import pandas as pd

def filter_documents(df):
    # Dictionary to store the results of each step
    result = {}
    
    # Total number of documents
    total_docs = len(df)
    remaining_docs = df  # Start with the full DataFrame
    
    print(f"Total documents: {total_docs}")
    
    # Sequential filtering
    for i in range(len(df.columns)):
        # Get the current column (filter step)
        current_filter = remaining_docs.iloc[:, i]  # Current filter column
        
        # Identify rows to eliminate and remaining rows
        eliminated_docs = remaining_docs[current_filter == True]  # Rows to eliminate
        remaining_docs = remaining_docs[current_filter != True]  # Remaining rows
        
        # Store the counts for reporting
        result[f"step_{i+1}: {df.columns[i]}"] = {
            "eliminated_count": len(eliminated_docs),
            "remaining_count": len(remaining_docs)
        }
    
    # Final remaining documents after all steps
    final_docs = len(remaining_docs)
    print(f"Final remaining documents: {final_docs}")
    
    # Print the results for each step
    for step, values in result.items():
        print(f"{step}: {values['eliminated_count']}, Remaining = {values['remaining_count']}")
        
    result["Total documents"]=total_docs
    return remaining_docs, result



remaining_docs, result=filter_documents(df_final2.iloc[:,11:-3])

result=pd.DataFrame(result)

Total documents: 3231
Final remaining documents: 1016
step_1: 1. eliminated because no pdf: 385, Remaining = 2846
step_2: 2. eliminated because corrupt file or no keyword: 410, Remaining = 2436
step_3: 3. eliminated because editorial retraction: 59, Remaining = 2377
step_4: 4. eliminated because book reviews, etc.: 198, Remaining = 2179
step_5: 5. eliminated because scanned with other relevant doc, duplicate or other technical issue: 228, Remaining = 1951
step_6: 6. eliminated because context/meaning: 854, Remaining = 1097
step_7: 7. eliminated because before 1960: 81, Remaining = 1016


In [234]:
pd.DataFrame(result).to_json("2025-06-07 filtering_results.json")

In [53]:
import pandas as pd
result=pd.read_json("2025-06-07 filtering_results.json")

In [236]:
df

,ID,Date,Type,Doi,Year,journal,Type Recoded,confirmed_2024,7. eliminated because before 1960
0,Nature1,2023-06-22,News,10.103...,2023,Nature,Journa...,True,False
1,Nature10,2023-04-27,Resear...,10.103...,2023,Nature,Other,False,False
2,Nature100,2021-06-09,Commen...,10.103...,2021,Nature,Journa...,True,False
3,Nature...,1926-12-11,News,10.103...,1926,Nature,Contri...,False,True
4,Nature...,2005-09-28,News,10.103...,2005,Nature,Contri...,False,False
...,...,...,...,...,...,...,...,...,...
3224,Scienc...,1963-09-27,Perspe...,10.112...,1963,Science,Contri...,False,False
3225,Scienc...,2001-09-21,Perspe...,10.112...,2001,Science,Contri...,False,False
3226,Scienc...,1918-03-01,Perspe...,10.112...,1918,Science,Contri...,False,True
3227,Scienc...,1918-03-01,Perspe...,10.112...,1918,Science,Contri...,False,True


In [47]:
df_final2.columns

Index(['ID', 'Doi', 'journal', 'Title', 'Date', 'Year', 'Type', 'Type Recoded',
       'confirmed_2024', 'PDF retrieved', 'has_retraction',
       '1. eliminated because no pdf',
       '2. eliminated because corrupt file or no keyword',
       '3. eliminated because editorial retraction',
       '4. eliminated because book reviews, etc.',
       '5. eliminated because scanned with other relevant doc, duplicate or other technical issue',
       '6. eliminated because context/meaning',
       '7. eliminated because before 1960', '3-7 eliminated',
       '8. retained for analysis', '9. retained for analysis (revised)'],
      dtype='object')

In [77]:
df_final2[['ID', 'Doi','Year','1. eliminated because no pdf',
       '2. eliminated because corrupt file or no keyword',
       '3. eliminated because editorial retraction',
       '4. eliminated because book reviews, etc.',
       '5. eliminated because scanned with other relevant doc, duplicate or other technical issue',
       '6. eliminated because context/meaning',
       '7. eliminated because before 1960',"9. retained for analysis (revised)"]].to_excel("2025-06-25 Meta_data full.xlsx")

In [240]:
# manually split two texts: These PDFs have two seperate relevant texts on retraction in one file.

df_txt=pd.read_json("2025-04-03 PDF Import complete.json").dropna(subset="texts") # This file contains all text. the later one only filtered text.

ida1=df_txt.loc['Nature507','neighboring_texts'].index('GM study retracted')
ida2=df_txt.loc['Nature507','neighboring_texts'].index('Bowing to scientists nearuniversal scorn,Food and Chemical Toxicology has retracted a controversial paper(G.-E.Séralini et al. FoodChem.Toxicol.50, 4221-4231; 2012) thatclaimed that Monsanto\'s genetically modified(GM)maize (corn) causes serious disease in rats.The authorshad earlier refusedtowithdrawit.The paper showed"no evidence of fraud or intentional misrepresentation of the data", said a 28 November statement frompublishers Elsevier, but the small number and type of animals used in the study meanthat"no definitive conclusions can be reached" See go.nature.com/fk2auz for more.')
idb1=df_txt.loc['Nature507','neighboring_texts'].index('Dance retraction')
idb2=df_txt.loc['Nature507','neighboring_texts'].index('An eight-year-old Nature paper that reported a strong correlation between the symmetry of Jamaican dancers and their dancing ability was retracted on 27November (W.M.Brownetal.Nature http://doi.org/p9m; 2013).No reason is given in the retraction notice, but in April co-author Robert Trivers, an evolutionary biologist at Rutgers University inNewBrunswick,New Jersey, publicly released a university investigation report alleging that then-postdoc and co-author William Brown had faked data for the paper (see Nature 497,170-171;2013). Brown disputed the findings. Trivers has sought since 2008 to retract the paper, which implied that dancing ability might have evolved under sexual selection in humans.')

idc1=df_txt.loc['Nature454','neighboring_texts'].index("Study retracted")
idc2=df_txt.loc['Nature454','neighboring_texts'].index("A landmark paper proposing a link between an influenza vaccine and narcolepsy was retracted on 31July.The study, publishedlast year (A.K.Dela Herran-Arita et al. Sci.Transl.Med.5,216ra176; 2013),reported that some people with narcolepsy had immune cells that target a wakefulness-maintaining neurotransmitter.The cells also recognize some components of flu vaccines, it said, and the results explainedwhy some children in Europe developed narcolepsy after receiving a vaccine forH1N1 swine flu. Butthe teamdiscovered that it could not reproduce its own findings,and so retracted the paper.See go.nature.com/ hbrrvi for more.")
ide=df_txt.loc['Nature454','neighboring_texts'].index('types of mature cells. In 2011,he stunned the world by mimicking an early stage in the development of the eye in vitro using embryonic stem cells.But over the past six months he became caught up in the controversy surroundingtwoNature papers that claimed embryonic stem cells could be created through a method called stimulus-triggered acquisition of pluripotency (STAP). The papers were retracted on 2 July after evidence of misconduct was found.Sasai, a co-author, was cleared of directinvolvementbutwas criticized for poor oversight of research."The world scientific community has lost an irreplaceable scientist" said RIKEN president Ryoji Noyori. See go.nature.com/ etrboi for more.')

texts_nature454 = df_txt.loc['Nature454', 'neighboring_texts']
texts_nature507 = df_txt.loc['Nature507', 'neighboring_texts']

df_txt.loc['Nature507b',:]=df_txt.loc['Nature507',:]
df_txt.loc['Nature454b',:]=df_txt.loc['Nature454',:]

df_txt["text"]=df_txt['neighboring_texts'].str.join(" ")

df_txt.loc['Nature507b','text']='. '.join([texts_nature507[i] for i in [ida1, ida2]])
df_txt.loc['Nature507','text']='. '.join([texts_nature507[i] for i in [idb1, idb2]])

df_txt.loc['Nature454b','text']='. '.join([texts_nature454[i] for i in [ide]])
df_txt.loc['Nature454','text']='. '.join([texts_nature454[i] for i in [idc1, idc2]])

df_txt.loc[['Nature507','Nature507b','Nature454','Nature454b'],"text"]

Nature507     Dance ...
Nature507b    GM stu...
Nature454     Study ...
Nature454b    types ...
Name: text, dtype: object

In [242]:
df_txt[['path', 'folder', 'journal','labels', 'retraction_counts','text']].to_json("2025-06-06 PDF import filtered.json")

In [72]:
# This is not applied later -- It creates alternative text columns from the PDF import

#from MyLib import labelfilter

# Apply the functions to create new columns

#df['filtered_texts'] = df.apply(lambda row: labelfilter.filter_texts(row['texts'], row['retraction_counts']), axis=1)

#df['neighboring_texts'] = df.apply(lambda row: labelfilter.get_neighboring_texts(row['texts'], row['retraction_counts']), axis=1)
#df['texts_until_section_header'] = df.apply(lambda row: labelfilter.get_texts_until_section_header(row['texts'], row['labels'], row['retraction_counts']), axis=1)
#df[['texts', 'filtered_texts','neighboring_texts','texts_until_section_header']].apply(lambda col: col.apply(len).mean())

# Create the flowchart

In [43]:
result

,step_1: 1. eliminated because no pdf,step_2: 2. eliminated because corrupt file or no keyword,step_3: 3. eliminated because editorial retraction,"step_4: 4. eliminated because book reviews, etc.","step_5: 5. eliminated because scanned with other relevant doc, duplicate or other technical issue",step_6: 6. eliminated because context/meaning,step_7: 7. eliminated because before 1960,Total documents
eliminated_count,385,410,59,198,228,854,81,3231
remaining_count,2846,2436,2377,2179,1951,1097,1016,3231


In [244]:
# New keys for the dictionary
new_keys = {
    'step_1: 1. eliminated because no pdf': 'Step 1: No PDF available',
    'step_2: 2. eliminated because corrupt file or no keyword': 'Step 2: Corrupt file or missing keyword',
    'step_3: 3. eliminated because editorial retraction': 'Step 3: Editorial retraction',
    'step_4: 4. eliminated because book reviews, etc.': 'Step 4: Excluded as book reviews or similar',
    'step_5: 5. eliminated because scanned with other relevant doc, duplicate or other technical issue': 'Step 5: False article on page or duplicate',
    'step_6: 6. eliminated because context/meaning': 'Step 6: Excluded due to irrelevant context',
    'step_7: 7. eliminated because before 1960': 'Step 7: Published before 1960',
    'Total documents': 'Total documents'
}

# Create a new dictionary with renamed keys
renamed_result = {new_keys[key]: value for key, value in result.items()}

renamed_result

{'Step 1: No PDF available': eliminated_count     385
 remaining_count     2846
 Name: step_1: 1. eliminated because no pdf, dtype: int64,
 'Step 2: Corrupt file or missing keyword': eliminated_count     410
 remaining_count     2436
 Name: step_2: 2. eliminated because corrupt file or no keyword, dtype: int64,
 'Step 3: Editorial retraction': eliminated_count      59
 remaining_count     2377
 Name: step_3: 3. eliminated because editorial retraction, dtype: int64,
 'Step 4: Excluded as book reviews or similar': eliminated_count     198
 remaining_count     2179
 Name: step_4: 4. eliminated because book reviews, etc., dtype: int64,
 'Step 5: False article on page or duplicate': eliminated_count     228
 remaining_count     1951
 Name: step_5: 5. eliminated because scanned with other relevant doc, duplicate or other technical issue, dtype: int64,
 'Step 6: Excluded due to irrelevant context': eliminated_count     854
 remaining_count     1097
 Name: step_6: 6. eliminated because context

In [246]:
from graphviz import Digraph

def create_flowchart(result):
    # Create a new directed graph
    flowchart = Digraph()
    #flowchart.attr(rankdir='LR') 
    flowchart.attr(rankdir='LR', nodesep='0.1', ranksep='0.5',size='22,5')  # Adjusted spacing for a more compact layout

    # Extract total documents and final remaining documents
    total_docs = result['Total documents']['remaining_count']
    final_remaining = result['Step 7: Published before 1960']['remaining_count']

    # Add the total documents node
    flowchart.node('A', f'Total Documents: \n{total_docs}', shape='ellipse', style='filled', fillcolor='lightblue', fontsize='12', fontname="Helvetica")

    # Add nodes for each step
    prev_node = 'A'
    for i, (step, values) in enumerate(result.items()):
        if step == 'Total documents':  # Skip the total documents key
            continue
        eliminated = values['eliminated_count']
        remaining = values['remaining_count']
        current_node = f'Step{i+1}'
        step_description = step.split(": ")[1]  # Extract the step description

        flowchart.node(
            current_node,
            f'{step_description}\nEliminated: {eliminated} \n Remaining: {remaining}',
            shape='box',
            style='rounded,filled',
           # fillcolor='lightgrey',
            fontsize='12',  # Smaller font size for clarity
            fontname="Helvetica")
        
        flowchart.edge(prev_node, current_node)
        prev_node = current_node

    # Add the final remaining documents node
        flowchart.node('Final', f'Final Remaining \n Documents: \n{final_remaining}', shape='ellipse', style='filled', fillcolor='lightgreen', 
                   fontsize='12', fontname='Helvetica-Bold')
        
    flowchart.edge(prev_node, 'Final')

    # Render the flowchart to a file and display it
    flowchart.render('flowchart3', format='pdf', cleanup=True)
    flowchart.view()



# Call the function to create the flowchart
create_flowchart(renamed_result)


In [164]:
pd.DataFrame(renamed_result)

,Step 1: No PDF available,Step 2: Corrupt file or missing keyword,Step 3: Editorial retraction,Step 4: Excluded as book reviews or similar,Step 5: False article on page or duplicate,Step 6: Excluded due to irrelevant context,Step 7: Published before 1960,Total documents
eliminated_count,385,410,59,198,226,854,81,3231
remaining_count,2846,2436,2377,2179,1953,1099,1018,3231
